In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import BartTokenizer, BartForConditionalGeneration, DataCollatorForSeq2Seq, Trainer, TrainingArguments

# Load your data
df = pd.read_csv("slogan_final_dataset.csv")
df.dropna(subset=['company', 'description', 'slogan'], inplace=True)

# Prepare the dataset by adding a combined text column and labels
df['text'] = df['company'] + " described as: " + df['description']
df['labels'] = df['slogan']

# Split data into training and evaluation sets
train_df, eval_df = train_test_split(df, test_size=0.1)

# Create datasets
train_dataset = Dataset.from_pandas(train_df[['text', 'labels']])
eval_dataset = Dataset.from_pandas(eval_df[['text', 'labels']])

2024-05-02 23:03:01.431121: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


FileNotFoundError: [Errno 2] No such file or directory: 'kaggle.csv'

In [ ]:
# Initialize tokenizer and model
model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Define tokenization and encoding function for datasets
def tokenize_function(examples):
    model_inputs = tokenizer(examples['text'], max_length=512, truncation=True, padding="max_length")
    # Tokenize the labels as well
    labels = tokenizer(examples['labels'], max_length=512, truncation=True, padding="max_length").input_ids
    model_inputs['labels'] = labels
    return model_inputs

# Apply function to datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/4509 [00:00<?, ? examples/s]

Map:   0%|          | 0/501 [00:00<?, ? examples/s]

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./bartresults',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=50,
    eval_steps=50,
    save_steps=50,
    warmup_steps=500,
    prediction_loss_only=True,
    fp16=True  # Assuming CUDA and compatible GPU
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained('./final_bart_model')
tokenizer.save_pretrained('./final_bart_model')

NameError: name 'TrainingArguments' is not defined

In [9]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

# Function to load the model and tokenizer
def load_model(model_path):
    tokenizer = BartTokenizer.from_pretrained(model_path)
    model = BartForConditionalGeneration.from_pretrained(model_path)
    return model, tokenizer

# Function to generate a slogan
def generate_slogan(model, tokenizer, company_name, description, device='cuda'):
    # Combine company name and description into a single input text
    input_text = f"{company_name} described as: {description}"
    
    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    
    # Generate output ids, correctly using the temperature parameter
    output_ids = model.generate(
        input_ids,
        max_length=80,
        num_beams=5,
        early_stopping=True,
        do_sample=True,
        temperature=3.0,  # Ensures some creativity
        top_k=40,
    )
    
    # Decode the output ids to a string
    slogan = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return slogan

# Example usage
if __name__ == '__main__':
    # Path to your trained model
    model_path = './final'
    
    # Load model and tokenizer
    model, tokenizer = load_model(model_path)
    
    # Ensure the model is on the appropriate device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    
    # Input your own company name and description
    company_name = "SkyCompute Technologies"
    description = "SkyCompute Technologies pioneers advanced cloud solutions to streamline business operations and enhance data management. Our flagship product, the 'SkyNode', integrates AI capabilities seamlessly into your infrastructure, supported by our commitment to security, performance, and eco-friendly practices. We empower organizations to innovate and scale efficiently, ensuring a robust digital transformation journey."
    
    company2_name = "Tesla"
    company2_description = "Tesla is a pioneering force in the automotive industry, recognized for its cutting-edge electric vehicles (EVs) and renewable energy solutions. Founded by Elon Musk in 2003, Tesla has revolutionized transportation with its sleek and high-performance electric cars, including the Model S, Model 3, Model X, and Model Y. Beyond automobiles, Tesla is leading the charge in renewable energy innovation, with products like the Powerwall and Solar Roof, reshaping the way we power our homes and businesses. With a focus on sustainability, innovation, and pushing the boundaries of what's possible, Tesla is driving the future of mobility and clean energy worldwide."

    # Generate slogan
    slogan = generate_slogan(model, tokenizer, company2_name, company2_description, device)
    print('Generated Slogan:', slogan)


Generated Slogan: Tesla - the leading electric car company
